In [ ]:
import json
import os
from typing import Tuple, Union

from science_jubilee.labware.Labware import Labware, Location, Well
from science_jubilee.tools.Tool import (
    Tool,
    ToolConfigurationError,
    ToolStateError,
    requires_active_tool,
)

In [ ]:
class MixingSyringe(Tool):
    """
    A single‑motor syringe capable of both aspirating and dispensing,
    plus mixing, all controlled by one stepper axis.
    """

    def __init__(self, index: int, name: str, config: str):
        """
        :param index: tool index on the machine
        :param name:  human‑readable tool name
        :param config: name of a JSON file (without “.json”) in configs/
        """
        super().__init__(index, name)

        # to be loaded from config:
        self.unit = None          # e.g. "milliliters"
        self.max_volume = None    # in unit
        self.min_volume = None    # in unit
        self.min_range = None     # in mm
        self.max_range = None     # in mm
        self.mm_to_ml = None      # conversion factor

        self.load_config(config)

    def load_config(self, config: str):
        """
        Load and validate JSON config from configs/{config}.json

        Expected keys:
          - unit       (str)
          - max_volume (float)
          - min_volume (float)
          - min_range  (float)
          - max_range  (float)
          - mm_to_ml   (float)
        """
        cfg_dir = os.path.join(os.path.dirname(__file__), "configs")
        cfg_path = os.path.join(cfg_dir, f"{config}.json")
        if not os.path.isfile(cfg_path):
            raise ToolConfigurationError(f"Config file not found: {cfg_path}")

        with open(cfg_path, "r") as f:
            data = json.load(f)

        required = ["unit", "max_volume", "min_volume", "min_range", "max_range", "mm_to_ml"]
        missing = [k for k in required if k not in data]
        if missing:
            raise ToolConfigurationError(f"Missing config keys: {missing}")

        self.unit       = data["unit"]
        self.max_volume = data["max_volume"]
        self.min_volume = data["min_volume"]
        self.min_range  = data["min_range"]
        self.max_range  = data["max_range"]
        self.mm_to_ml   = data["mm_to_ml"]


    def post_load(self):
      """Bind the single extruder axis letter for this syringe."""
      tool_info = json.loads(self._machine.gcode('M409 K"tools[]"'))["result"]
      for tool in tool_info:
        if tool["number"] == self.index:
          # Syringe tool has only one extruder channel
          self.e_drive = f"E{tool['extruders'][0]}"
          break
      else:
        raise ToolStateError(f"Could not find tool index {self.index} in machine tools")


    def check_bounds(self, pos):
        """Disallow commands outside of the syringe's configured range

        :param pos: The E position to check
        :type pos: float
        """

        if pos > self.max_range or pos < self.min_range:
            raise ToolStateError(f"Error: {pos} is out of bounds for the syringe!")

    @requires_active_tool
    def _aspirate(self, vol: float, s: int = 2000):
        """Plunger retract: pull in `vol` mL (negative mm move)."""
        de = -vol * self.mm_to_ml
        pos = float(self._machine.get_position()[self.e_drive]) + de
        self.check_bounds(pos)
        self._machine.move(de=de, wait=True)

    @requires_active_tool
    def _dispense(self, vol: float, s: int = 2000):
        """Plunger advance: push out `vol` mL (positive mm move)."""
        de = vol * self.mm_to_ml
        pos = float(self._machine.get_position()[self.e_drive]) + de
        self.check_bounds(pos)
        self._machine.move(de=de, wait=True)

    @requires_active_tool
    def aspirate(
        self,
        vol: float,
        location: Union[Well, Tuple[float, float, float], Location],
        s: int = 2000,
    ):
        """Move to `location` and aspirate `vol` mL."""
        x, y, z = Labware._getxyz(location)
        self._machine.safe_z_movement()
        self._machine.move_to(x=x, y=y)
        self._machine.move_to(z=z)
        self._aspirate(vol, s=s)

    @requires_active_tool
    def dispense(
        self,
        vol: float,
        location: Union[Well, Tuple[float, float, float], Location],
        s: int = 2000,
    ):
        """Move to `location` and dispense `vol` mL."""
        x, y, z = Labware._getxyz(location)
        self._machine.safe_z_movement()
        self._machine.move_to(x=x, y=y)
        self._machine.move_to(z=z)
        self._dispense(vol, s=s)

    @requires_active_tool
    def mix(
        self,
        vol: float,
        location: Union[Well, Tuple[float, float, float], Location],
        repetitions: int,
        s: int = 2000,
    ):
        """
        Mix by repeatedly aspirating and dispensing `vol` mL at `location`.

        :param vol: volume (mL) to move each cycle
        :param location: the well (or coords) to mix in
        :param repetitions: number of aspirate/dispense cycles
        :param s: speed (mm/min) for plunger moves
        """
        # get well coords
        x, y, z = Labware._getxyz(location)
        # slight offsets for bottom and top to avoid collisions
        z_asp = z + 1
        # we can pull z_top from the Well object if available
        if isinstance(location, Well):
            z_top = location.top_ + 1
        else:
            # fallback: lift 5 mm above bottom
            z_top = z + 5

        # travel to X/Y once
        self._machine.safe_z_movement()
        self._machine.move_to(x=x, y=y)

        for _ in range(repetitions):
            # aspirate at bottom
            self._machine.move_to(z=z_asp)
            self._aspirate(vol, s=s)
            # dispense at top
            self._machine.move_to(z=z_top)
            self._dispense(vol, s=s)